<a href="https://colab.research.google.com/github/SYEON9/radar_image_predict/blob/main/radar_image_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#drive mount
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import zipfile


import glob
import torch
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, BatchNormalization, concatenate, ConvLSTM2D, Conv3D
from tensorflow.keras import Model, layers, optimizers

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#data load
train_file = zipfile.ZipFile('/content/drive/MyDrive/235646_공공데이터 활용 수력 댐 강우예측 AI 경진대회_data/train.zip')
train_file.extractall('.')

In [ ]:
#이미지 가져오기
#train데이터의 이름을 train_files라는 변수에 하나의 리스트로 넣음.
train_files = glob.glob('/content/train/*.npy')
len(train_files) #전체 데이터 개수 확인

# 새 섹션

In [ ]:
#1번째 데이터만 불러와서 데이터를 확인하자.
data_1st = np.load(train_files[0])
data_1st.shape

In [ ]:
#시각화해서 다시 확인하자. 
#color map 지정
color_map = plt.cm.get_cmap('RdBu')  #"RdBu"색으로 이미지를 칠함. 
color_map = color_map.reversed()  #지정된 색을 뒤집는다. 

#시각화
plt.style.use('fivethirtyeight')  #스타일 시트 지정.
plt.figure(figsize = (20,20)) #창의 크기를 (20, 20)으로 설정. 

for i in range(4):
    plt.subplot(1, 5, i+1)
    plt.imshow(data_1st[:,:,i], cmap = color_map)

plt.subplot(1, 5, 5)
plt.imshow(data_1st[:,:,-1], cmap=color_map)
plt.show()

#데이터는 10분 간격의 5개의 이미지로 구성되어 있다. 
#그러나 마지막 데이터는 label이므로 모델을 학습할 때에는 앞의 4개만 사용해야 한다. 

# 새 섹션

In [ ]:
#데이터 불러오기
#5개의 이미지 데이터를 4개의 train 데이터와 1개의 label 데이터로 분리한다. 
def trainGenerator():
    for file in train_files:
        dataset = np.load(file)
        target= dataset[:,:,-1].reshape(1,1,120,120,1)
        remove_minus = np.where(target < 0, 0, target)
        feature = dataset[:,:,:4].reshape(1,1,120,120,4)

        yield (feature, remove_minus)
        
#메모리 문제로 1개의 batch만큼만 올려 처리한다. 
#이미지 3채널을 5채널로 변경
#batch size를 256으로 설정. 
#prefetch를 사용하여 미리 데이터를 가져옴. -> 메모리 대기시간을 줄일 수 있음.
train_data = tf.data.Dataset.from_generator(trainGenerator, (tf.float32, tf.float32), (tf.TensorShape([1,1,120,120,4]),tf.TensorShape([1,1,120,120,1])))
train_dataset = train_data.batch(256).prefetch(1)
train_d = train_data

In [ ]:
def trainGenerator2():
    for file in train_files:
        dataset = np.load(file)
        target= dataset[:,:,-1].reshape(1,1,120,120,1)
        remove_minus = np.where(target < 0, 0, target)
        feature = dataset[:,:,:4].reshape(1,1,120,120,4)

        yield (feature, remove_minus)


In [ ]:
def trainGenerator():
    for file in tr_file:
        dataset = np.load(file)
        target= dataset[:,:,-1].reshape(1,1,120,120,1)        
        remove_minus = np.where(target < 0, 0, target)
        feature = dataset[:,:,:4].reshape(1,1,120,120,4)



        yield (feature, remove_minus)

def valGenerator():
    for file in val_file:
        dataset = np.load(file)
        target= dataset[:,:,-1].reshape(1,1,120,120,1)        
        remove_minus = np.where(target < 0, 0, target)
        feature = dataset[:,:,:4].reshape(1,1,120,120,4)


        yield (feature, remove_minus)

# 모델링

In [ ]:
#model
inp = layers.Input(shape = (1,120,120,4))

x = layers.ConvLSTM2D(64, kernel_size=(5,5), padding = 'same', return_sequences = True, activation = 'relu',)(inp)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(64, kernel_size=(5,5), padding = 'same', return_sequences = True, activation = 'relu',)(x)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(64, kernel_size=(5,5), padding = 'same', return_sequences = True, activation = 'relu',)(x)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(64, kernel_size=(5,5), padding = 'same', return_sequences = True, activation = 'relu',)(x)
x = layers.BatchNormalization()(x)

x = layers.Conv3D(filters = 1, kernel_size = (5,5,5), activation = 'sigmoid', padding='same')(x)



In [ ]:
model = keras.models.Model(inp,x)
model.compile(loss = 'mae', optimizer = 'adam')

In [ ]:
model.summary()

# 모델 훈련

In [ ]:
#필요한 라이브러리
import gc
from sklearn.model_selection import KFold

In [ ]:
#k-fold cross validation 
kf = KFold(n_splits=5, random_state=42)
for fold, (train, val) in enumerate(kf.split(train_files)):
    val_file = train_files[val]
    tr_file = train_files[train]

    train_dataset = tf.data.Dataset.from_generator(trainGenerator, (tf.float32, tf.float32), (tf.TensorShape([1,1,120,120,4]),tf.TensorShape([1,1,120,120,1])))
    train_dataset = train_dataset.shuffle(256)

    train_dataset = train_dataset.batch(128).prefetch(1)

    val_dataset = tf.data.Dataset.from_generator(valGenerator, (tf.float32, tf.float32), (tf.TensorShape([120,120,4]),tf.TensorShape([120,120,1])))
    val_dataset = val_dataset.batch(128).prefetch(1)


    callbacks = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.2, patience=2, verbose=0, mode='min',
        min_delta=0.0001, cooldown=0, min_lr=0
    )

    sv = tf.keras.callbacks.ModelCheckpoint(
        os.path.join(root_path,f'{fold}-rain.h5'), monitor='val_loss', verbose=0, save_best_only=True,
        save_weights_only=True, mode='min', save_freq='epoch')
    
    model.fit(train_dataset, epochs = 1, verbose=1, validation_data=val_dataset, callbacks=[callbacks, sv])

    
    model.save('/content/convlstm_model_'+str(idx)+'_lastEpoch.h5')
    idx += 1

   scores = model.evaluate(val_dataset, batch_size = BATCH_SIZE)
   all_scores.append(scores)

   pred = model.predict(test_x)


In [ ]:
t = trainGenerator2()

In [ ]:
t.__dir__()

In [ ]:
t.__sizeof__()

In [ ]:
def trainGenerator2():
    for file in train_files:
        dataset = np.load(file)
        target= dataset[:,:,-1].reshape(1,1,120,120,1)
        remove_minus = np.where(target < 0, 0, target)
        feature = dataset[:,:,:4].reshape(1,1,120,120,4)

        yield (feature, remove_minus)


In [ ]:
#generator를 사용하지 않고 데이터를 불러오기.

train_x, train_y = []

for file in train_files:  #mapping하는게 좋을까?
    dataset = np.load(file)
    target = dataset[:,:,-1].reshape(1,1,120,120,1)
    remove_minus = np.where(target<0,0,target)
    feature = dataset[:,:,4].reshape(1,1,120,120,4)

    train_x,append(feature)
    train_y.append(feature)


print(train_x.shape)
print(train_y.shape)

In [ ]:
#train, test 데이터로 슬라이싱 할 정수 만들기
def permutation_train_test_split(x, y, test_size=0.2, shuffle = True, random_state = 1004):
    test_num = int(x.shape[0]*test_size)
    train_num = x.shape[0] - test_num

test_num =  random.sample(train_files, int(len(train_files)*0.3))
test_x = train_x[test_num]
test_y = train_y[test_num]

In [ ]:
train_sample = []
for file in train_files